# GOMS 데이터 회귀 분석 - 직업군 예측

In [3]:
# !pip install openpyxl
# !pip install xlrd

In [4]:
import pandas as pd
import numpy as np
import openpyxl
import xlrd

In [5]:
data = pd.read_excel('./goms_filtered_re.xlsx', engine='openpyxl') # 각자 경로로 바꾸기

In [6]:
# null 확인
data.isnull().sum()

schoolType             0
schoolType_str         0
major                  0
graduY                 0
age                    0
internExp           5471
toeicScore         12757
tospGrade          16976
trainingHr         14182
jobseekYN              0
certNum             7813
mainJobCate         4868
mainJobCate_str     4868
midJobCate          4868
midJobCate_str      4868
subJobCate          4868
subJobCate_str      4868
dtype: int64

In [7]:
# 결측치 0으로 대체
data = data.fillna(0)

## 종속변수가 main

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

In [9]:
data_main = data.drop(['schoolType_str','subJobCate', 'mainJobCate_str', 'midJobCate', 'midJobCate_str', 'subJobCate_str'],axis=1)

In [10]:
X_ohe1 = pd.get_dummies(data_main, columns=['major'])
y_main = X_ohe1['mainJobCate']
x_main = X_ohe1.drop(['mainJobCate'],axis=1)

X_train, X_test, y_train, y_test = train_test_split(x_main, y_main, test_size=0.3, random_state=0)
rfr = RandomForestRegressor(max_depth = 3,random_state=12)
rfr.fit(X_train, y_train)

y_train_pred = rfr.predict(X_train)
train_mse = mean_squared_error(y_train, y_train_pred)
print("Train MSE: %.4f" % train_mse)
y_test_pred = rfr.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
print("Test MSE: %.4f" % test_mse)

Train MSE: 6.3319
Test MSE: 6.4864


## 종속변수가 mid

In [11]:
data_mid = data.drop(['schoolType_str','subJobCate', 'mainJobCate_str', 'mainJobCate', 'midJobCate_str', 'subJobCate_str'],axis=1)

In [12]:
X_ohe2 = pd.get_dummies(data_mid, columns=['major'])
y_mid = X_ohe2['midJobCate']
x_mid = X_ohe2.drop(['midJobCate'],axis=1)

X_train, X_test, y_train, y_test = train_test_split(x_mid, y_mid, test_size=0.3, random_state=0)
rfr = RandomForestRegressor(max_depth = 3,random_state=12)
rfr.fit(X_train, y_train)

y_train_pred = rfr.predict(X_train)
train_mse = mean_squared_error(y_train, y_train_pred)
print("Train MSE: %.4f" % train_mse)
y_test_pred = rfr.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
print("Test MSE: %.4f" % test_mse)

Train MSE: 538.1987
Test MSE: 552.0407
